# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

dotenv is to be able to read env file - where the api key is stored ↓

In [1]:
#%pip install python-dotenv
#%pip install openai

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.
This is a helper function for sending a prompt to an OpenAI chat model and getting back just the text of the response
- def get_completion(prompt, model=llm_model) → Defines a function named get_completion. It takes two arguments:
- prompt → the user’s input (string).
- model → defaults to the variable llm_model (set earlier in your code to "gpt-3.5-turbo" or similar).
- messages = … → OpenAI chat models expect a list of messages with roles ("system", "user", "assistant"). Here, we create a single-user message containing the prompt: [{"role": "user", "content": "Write a haiku about AI"}]
- response = … → Calls the OpenAI API using the ChatCompletion endpoint. Parameters:
model=model → which LLM to use (e.g. "gpt-3.5-turbo").
- messages=messages → the conversation history (just one message here).
- temperature=0 → makes the response more deterministic (less randomness).

- return … → The API returns a response object with a list of possible completions (choices).
- choices[0] → take the first completion.
- .message["content"] → grab just the text of the assistant’s reply.
Finally, return that text as the function’s output.

This function:
- Wraps your text into the format the OpenAI API expects.
- Sends it to the model you specify.
- Returns just the assistant’s reply text (not the whole JSON response) ↓


In [4]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [5]:
get_completion("What is 1+1?")

'1+1 equals 2.'

Course customer_email

In [6]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

My customer_email:

In [7]:
customer_email = """
this is super stupid \
do only stupid people work at \
your stupid company,\
your stupid lid \
fucked up my kitchen \
i want a frickin refund!
"""

In [8]:
style = """American English \
in a calm and respectful tone
"""

In [9]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
this is super stupid do only stupid people work at your stupid company,your stupid lid fucked up my kitchen i want a frickin refund!
```



In [10]:
response = get_completion(prompt)

In [11]:
response

'I am disappointed with the experience I had with your company. The lid I purchased ended up causing damage to my kitchen. I would like a refund for this issue. Thank you.'

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [12]:
#%pip install --upgrade langchain

### Model

Why use ChatOpenAI in LangChain?
- It standardizes the interface for different LLM providers (so your code looks the same whether you use OpenAI, Anthropic, Cohere, etc.).
- It makes it easy to plug models into LangChain chains, agents, and tools.
- It hides some of the boilerplate (messages=[...]) you had to write with the raw OpenAI API.


 This code creates a LangChain LLM object (chat) that wraps an OpenAI chat model (like GPT-3.5 or GPT-4) with deterministic behavior (temperature=0). You can now pass prompts/messages to it in a more structured way ↓


In [13]:
from langchain.chat_models import ChatOpenAI

In [14]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

### Prompt template

In [15]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

It lets you define prompts with variables (placeholders) that you can fill in later.
- .from_template(...) converts that string into a ChatPromptTemplate object.

The result is not plain text — it’s a template object you can use to format prompts safely and repeatedly ↓


In [16]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


- ↑ Instead of hardcoding full prompts, you can reuse and fill them with different inputs.
- This code takes a string with placeholders and turns it into a prompt template object that LangChain can use to reliably generate prompts for LLMs.


- messages is a list of message templates inside your chat prompt.
- Each element corresponds to a role in the conversation (like “system”, “user”, or “assistant”).
- In this case, since you built it from a simple string, you’ll likely just have one user message ↓


In [17]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

- prompt_template = a ChatPromptTemplate (container for message templates).
- prompt_template.messages[0] = the first message template (likely a HumanMessagePromptTemplate).
- .prompt = the underlying PromptTemplate object (the raw text with {sentence}).
- .input_variables = Every PromptTemplate keeps track of which placeholders (variables inside {}) are required to format the string.
- This code asks the first message template which placeholders it needs. It’s basically saying:
- “Hey template, what variables do I need to fill in before you’re ready to send to the LLM?”
- Lets you confirm what inputs you must provide when calling .format_messages().
- Prevents errors — if you forget to supply a required variable, LangChain will raise an exception ↓



In [18]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

1. Define style and the customer’s email ↓


In [19]:
customer_style = """American English \
in a calm and respectful tone
"""

Course customer_email:

In [20]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

My customer_email:

In [21]:
customer_email = """
this is super stupid \
do only stupid people work at \
your stupid company,\
your stupid lid \
fucked up my kitchen \
i want a frickin refund!
"""

2. Format the template with variables
- earlier you had a ChatPromptTemplate with placeholders like {style} and {text}.
- This call fills in those placeholders with customer_style and customer_email.
- The result: a list of LangChain Message objects (like HumanMessage) ↓



In [22]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [23]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [24]:
print(customer_messages[0])

content='Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nthis is super stupid do only stupid people work at your stupid company,your stupid lid fucked up my kitchen i want a frickin refund!\n```\n' additional_kwargs={} example=False


3. Send the message to the LLM
- Calls your ChatOpenAI instance (chat) with the filled messages.
- The model rewrites the pirate rant in calm/respectful American English.
- .content prints just the text reply ↓



In [25]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [26]:
print(customer_response.content)

I am disappointed with the quality of your product. The lid that I purchased from your company caused damage to my kitchen. I would like a refund for this issue. Thank you.


Course service_reply:

In [27]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

My service_reply:

In [28]:
service_reply = """Hey there stupid, \
who you calling stupid \
do you think I like working here \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
you will not be getting any refunds any time soon. \
Tough luck sucker!
"""

(course)A new desired service agent response style ↓

In [29]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

(my) desired service agent response style:

In [30]:
service_style_pirate = """\
a polite tone \
that speaks in nice English\
warm and friendly\
"""

Fills the template again ↓

In [31]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in nice Englishwarm and friendly. text: ```Hey there stupid, who you calling stupid do you think I like working here because it's your fault that you misused your blender by forgetting to put the lid on before you will not be getting any refunds any time soon. Tough luck sucker!
```



The LLM rewrites the rude service message into a polite pirate-style response ↓

In [32]:
service_response = chat(service_messages)
print(service_response.content)

Hello there, my dear. I must kindly ask you to refrain from using such harsh language. It seems there may have been a misunderstanding regarding the blender. Unfortunately, refunds will not be possible at this time. Thank you for your understanding.


↑ This code shows how to:
1. Define a prompt template with placeholders for {style} and {text}.
2. Fill it with a customer complaint (pirate rant) and a desired tone (calm, respectful).
3. Use ChatOpenAI to translate the text into that tone.
4. Do the same with a service reply, translating it into polite pirate English.


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

- In Python, this is a dictionary (a data type that stores key–value pairs).
- In JSON, it’s an object (very similar — JSON is often used for saving or sending data).

In [33]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [35]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

1. This imports the ChatPromptTemplate class from LangChain.
- ChatPromptTemplate is used to build prompts for chat-based models (like GPT).
- Instead of writing long prompts directly as strings, you can make templates with placeholders.
2. create an prompt from a template
- from_template(...) takes a string template (e.g., with variables) and turns it into a LangChain ChatPromptTemplate object.
- review_template is a string variable defined earlier
- When you later format the template, you fill in {review_text} with actual input.
3. print the structure of the template ↓

In [36]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [37]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


In [38]:
type(response.content)

str

In [49]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
# response.content.get('gift')

### Parse the LLM output string into a Python dictionary

1. ResponseSchema
- A blueprint for one piece of output you expect from the model.
- You define a schema by naming the field and describing what should go inside
2. StructuredOutputParser
- takes a ResponseSchema and builds a parser
- parser gives the model instructions on how to format the output (usually JSON)
- parser parses the model's raw text into a Python dictionary ↓

In [39]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

- we want a field called gift (y/n)
- we want a field "delivery_days"
- field "price_value" ↓

In [40]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [41]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

- output_parser was created from your ResponseSchemas.
- When you call .get_format_instructions(), the parser generates special text instructions that you can insert into your prompt for the LLM.
- These instructions tell the LLM exactly how to format its answer — usually as JSON with the fields you defined ↓

In [42]:
format_instructions = output_parser.get_format_instructions()

- ↑ Instead of you writing long instructions manually, get_format_instructions() auto-generates them from your schema.
- You then add format_instructions into your PromptTemplate so the LLM knows exactly how to respond.

In [43]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


- ↑ LLMs normally return free-form text → messy for code.
- ResponseSchema + StructuredOutputParser force the model to answer in structured JSON, so your program can reliably use the results.

1. Define the Prompt Template (review_template_2 = """...""")
- Clear guidance for each field (gift, delivery_days, price_value).
- A placeholder {text} → where the actual customer review will go.
- A placeholder {format_instructions} → where the JSON formatting rules (from output_parser.get_format_instructions()) will be inserted
2. Turn Template into a ChatPromptTemplate
- ChatPromptTemplate is a LangChain helper for chat-style prompts.
- This lets you later substitute variables ({text}, {format_instructions}) with real values
3. Format the Prompt with Variables
- Here you fill in the blanks:
- {text} → replaced with the actual customer_review string.
- {format_instructions} → replaced with the formatting rules from your StructuredOutputParser.
- The result (messages) is a list of chat messages ready to send to an LLM ↓

In [45]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

- *messages* is a list of LangChain BaseMessage objects (usually HumanMessage, sometimes SystemMessage or AIMessage).
- Each message has two important parts:
    - role → who the message is from (human/system/AI).
    - content → the actual text of the message.
- *messages[0]* gets the first message in the list.
- In your case, that’s the human instruction prompt you just built.
- *.content* extracts just the text content of that message ↓

In [46]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "

- this line is where you actually send your prompt to the LLM ↓

In [47]:
response = chat(messages)

In [48]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


**Turning the LLM’s raw text output into a usable Python dictionary ↓**
- response.content - This is the raw string output from the LLM (likely JSON)
- output_parser.parse(...) - Converts that raw string into a Python dictionary
- output_dict - Stores the parsed dictionary so you can reuse it.

In [49]:
output_dict = output_parser.parse(response.content)

- In a Jupyter Notebook, the last line of a cell is automatically displayed as output.
- So instead of *print(output_dict)*, you can just write *output_dict* ↓

In [50]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [51]:
type(output_dict)

dict

- pull values out of the parsed dictionary
- .get('delivery_days')
    - .get() is a dictionary method in Python.
    - It looks for the key 'delivery_days' in the dictionary.
    - If found → returns the value.
    - If not found → returns None (instead of raising an error, which would happen if you used output_dict['delivery_days']).

In [52]:
output_dict.get('delivery_days')

2